In [1]:
from pathlib import Path
from typing import cast

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
import ml_project.data as d
import ml_project.helpers as h

In [2]:
ds = d.Dataset(Path("../data/15s.arff"))
ds.cls_id_repr_map

{1: 'BROWSING (1)',
 2: 'CHAT (2)',
 3: 'STREAMING (3)',
 4: 'MAIL (4)',
 5: 'VOIP (5)',
 6: 'P2P (6)',
 7: 'FT (7)',
 8: 'VPN-VOIP (8)',
 9: 'VPN-CHAT (9)',
 10: 'VPN-STREAMING (10)',
 11: 'VPN-FT (11)',
 12: 'VPN-BROWSING (12)',
 13: 'VPN-P2P (13)',
 14: 'VPN-MAIL (14)'}

In [6]:
ds._arff[0], ds._arff[1]
ds.orig

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,pps,...,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,cls
0,9368711.0,16.0,4.0,1564818.0,1549373.0,190205.285714,203290.456522,389822.391917,370323.719754,10.353612,...,267600.198443,1871488.0,1.983656e+06,2195089.0,1.832197e+05,1234883.0,1420565.0,1523088.0,161096.539275,2
1,7340238.0,18.0,4.0,1567554.0,1527893.0,165686.977273,186914.846154,317267.548742,304370.651301,11.580006,...,221462.862028,1491627.0,3.572433e+06,5653239.0,2.942704e+06,1131498.0,1324636.0,1517774.0,273138.379008,2
2,4644225.0,29.0,15.0,1270547.0,1079974.0,165865.178571,195302.130435,329473.126261,300492.588227,11.412022,...,217475.425246,1758922.0,1.758922e+06,1758922.0,0.000000e+00,1079974.0,1079974.0,1079974.0,0.000000,2
3,4978735.0,19.0,8.0,2492050.0,2457286.0,239543.250000,276596.388889,612435.304238,628339.573544,8.034169,...,436959.716436,1710925.0,2.382905e+06,3054885.0,9.503232e+05,1346073.0,1894031.5,2441990.0,774930.342317,2
4,11838189.0,19.0,10.0,3094089.0,3093543.0,243766.500000,295954.725000,599721.781709,625632.703972,7.602514,...,436129.639296,1747431.0,2.400446e+06,3240696.0,6.232744e+05,1394455.0,1983227.0,3042717.0,725987.829075,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18753,73240.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,27.307482,...,0.000000,-1.0,0.000000e+00,-1.0,0.000000e+00,-1.0,0.0,-1.0,0.000000,6
18754,52083.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,38.400246,...,0.000000,-1.0,0.000000e+00,-1.0,0.000000e+00,-1.0,0.0,-1.0,0.000000,6
18755,67923.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,29.445107,...,0.000000,-1.0,0.000000e+00,-1.0,0.000000e+00,-1.0,0.0,-1.0,0.000000,6
18756,313588.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,6.377795,...,0.000000,-1.0,0.000000e+00,-1.0,0.000000e+00,-1.0,0.0,-1.0,0.000000,6


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score

df = ds.flagged

X = df.drop(columns=["cls"])
y = df["cls"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

Test accuracy: 0.7993782004389174


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

df = ds.flagged

X = df.drop(columns=["cls"])
y = df["cls"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

for k in range(1, 50):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)

  y_pred = knn.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f"k={k}, accuracy={accuracy}")

k=1, accuracy=0.6644111192392099
k=2, accuracy=0.6441111923920995
k=3, accuracy=0.651609363569861
k=4, accuracy=0.6521580102414045
k=5, accuracy=0.653986832479883
k=6, accuracy=0.6519751280175567
k=7, accuracy=0.6494147768836869
k=8, accuracy=0.6442940746159473
k=9, accuracy=0.6389904901243599
k=10, accuracy=0.6399049012435991
k=11, accuracy=0.6393562545720556
k=12, accuracy=0.6371616678858815
k=13, accuracy=0.6399049012435991
k=14, accuracy=0.6366130212143379
k=15, accuracy=0.6358814923189466
k=16, accuracy=0.6340526700804682
k=17, accuracy=0.6346013167520117
k=18, accuracy=0.6298463789319678
k=19, accuracy=0.6318580833942941
k=20, accuracy=0.6298463789319678
k=21, accuracy=0.6267373811265545
k=22, accuracy=0.6289319678127286
k=23, accuracy=0.6247256766642283
k=24, accuracy=0.6247256766642283
k=25, accuracy=0.6236283833211412
k=26, accuracy=0.6217995610826628
k=27, accuracy=0.6230797366495977
k=28, accuracy=0.6221653255303584
k=29, accuracy=0.6199707388441843
k=30, accuracy=0.62216532